In [7]:
class AnyName:
    def __init__(self, currency):
        self.currency = currency
        self._secret = '123'
        self.__topsecret = '456'
    
    def method_1(self):
        self.currency = 'eur'
        
    def method_2(self):
        print(self.currency)
        
    @staticmethod
    def method_3(a, b):
        return a + b

In [11]:
class SecondName(AnyName):
    def __init__(self, currency):
        self.currency = currency
        super().__init__(currency=currency)
        
        
    def print(self):
        print(self.currency)

In [12]:
s = SecondName(currency='rub')

TypeError: __init__() missing 1 required positional argument: 'currency'

# Паттерн фабрика
- проверка корректности запросов
- использование определенного логина-пароля
- процесс записи в таблицы
- ...

In [ ]:
class FinanceReport(Composition(TrafficReport, CRMReport, EmailBase)):

In [3]:
second = SecondName()

In [5]:
second.currency

'usd'

In [6]:
second.print()

usd


In [29]:
a = AnyName()

In [30]:
a._secret

'123'

In [31]:
a.__topsecret

AttributeError: 'AnyName' object has no attribute '__topsecret'

In [11]:
a.method_3(1, 2)

3

In [3]:
b = AnyName()

In [14]:
a.currency

'usd'

In [27]:
a.method_1()

AttributeError: 'AnyName' object has no attribute '_AnyName__topsecret'

In [19]:
a.method_2()

rub


In [18]:
a.currency = 'rub'

In [5]:
b.method_1()

In [7]:
b.method_2()

eur


# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [ ]:
token = '9pq3htfqu3h4fi;qtp3uh4t'

In [13]:
import requests

In [14]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5

DEMO_COUNTER = 44147844

In [46]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, 
                 start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''
            
        self.dimensions = ''
        self.metrics = ''
        self.report = []
        
    def api_request(self, offset=1):
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        self.dimensions = 'ym:s:date'
        self.metrics = 'ym:s:visits'
        
        report = self.full_report()
        self.report = report
        
    def traffic_sources(self):
        self.dimensions = 'ym:s:date,ym:s:<attribution>TrafficSource'
        self.metrics = 'ym:s:visits'
        
        report = self.api_request()
        self.report = report
        
    def reformat_api_report(self):
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report = reformatted_report
        
    def full_report(self):
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

In [48]:
m = Metrika(start_date='2021-01-01', end_date='2021-01-31')

In [52]:
m.limit = 2

In [53]:
m.traffic()

Starting offset 1
Starting offset 3
Starting offset 5
Starting offset 7
Starting offset 9
Starting offset 11
Starting offset 13
Starting offset 15
Starting offset 17
Starting offset 19
Starting offset 21
Starting offset 23
Starting offset 25
Starting offset 27
Starting offset 29
Starting offset 31


In [41]:
m.traffic_sources()

In [42]:
m.report

[{'dimensions': [{'name': '2020-09-01'},
   {'icon_id': '2',
    'icon_type': 'traffic-source',
    'name': 'Переходы из поисковых систем',
    'id': 'organic'}],
  'metrics': [215.0]},
 {'dimensions': [{'name': '2020-09-09'},
   {'icon_id': '2',
    'icon_type': 'traffic-source',
    'name': 'Переходы из поисковых систем',
    'id': 'organic'}],
  'metrics': [211.0]},
 {'dimensions': [{'name': '2020-09-08'},
   {'icon_id': '2',
    'icon_type': 'traffic-source',
    'name': 'Переходы из поисковых систем',
    'id': 'organic'}],
  'metrics': [206.0]},
 {'dimensions': [{'name': '2020-09-02'},
   {'icon_id': '2',
    'icon_type': 'traffic-source',
    'name': 'Переходы из поисковых систем',
    'id': 'organic'}],
  'metrics': [205.0]},
 {'dimensions': [{'name': '2020-09-03'},
   {'icon_id': '2',
    'icon_type': 'traffic-source',
    'name': 'Переходы из поисковых систем',
    'id': 'organic'}],
  'metrics': [203.0]}]

In [43]:
m.reformat_api_report()

In [44]:
m.report

[['2020-09-01', 'Переходы из поисковых систем', 215.0],
 ['2020-09-09', 'Переходы из поисковых систем', 211.0],
 ['2020-09-08', 'Переходы из поисковых систем', 206.0],
 ['2020-09-02', 'Переходы из поисковых систем', 205.0],
 ['2020-09-03', 'Переходы из поисковых систем', 203.0]]

In [26]:
m.traffic()

In [27]:
m.report

[{'dimensions': [{'name': '2020-09-08'}], 'metrics': [530.0]},
 {'dimensions': [{'name': '2020-09-09'}], 'metrics': [526.0]},
 {'dimensions': [{'name': '2020-09-07'}], 'metrics': [511.0]},
 {'dimensions': [{'name': '2020-09-01'}], 'metrics': [505.0]},
 {'dimensions': [{'name': '2020-09-02'}], 'metrics': [505.0]}]

In [22]:
m.dimensions = 'ym:s:date'
m.metrics = 'ym:s:visits'

In [23]:
m.api_request()

[{'dimensions': [{'name': '2020-09-08'}], 'metrics': [530.0]},
 {'dimensions': [{'name': '2020-09-09'}], 'metrics': [526.0]},
 {'dimensions': [{'name': '2020-09-07'}], 'metrics': [511.0]},
 {'dimensions': [{'name': '2020-09-01'}], 'metrics': [505.0]},
 {'dimensions': [{'name': '2020-09-02'}], 'metrics': [505.0]}]